# Milestone 2: TF-IDF Search using Cosine Similarity

Import spacy and the small English model

In [ ]:
import spacy
import json
import os
nlp = spacy.load('en_core_web_sm')

In [ ]:
#Opens the json file at ./outputs/milestone-1/tokenized-data.json
with open('./outputs/milestone-1/tokenized-data.json') as f:
    data = json.load(f)

In [ ]:
#Creates a corpus of tokens from the json file
corpus = []
for item in data:
    for token in item['tokenized_text']:
        corpus.append(token)
corpus = set(corpus)